In [451]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')

# Create hexagon map of Switzerland

## QGIS -  Geographic Information System

**QGIS** is a great software to create and edit geospatial information. In order to create an hexagon map of Switzerland, we used the **MMQGIS** plugin which facilitates manipulating vector map layers. We proceded as follows:
* *Obtain Swtizerland bounderies*: bounderies of all countries can be obtained from the shape file *50m_admin_0_countries.shp* from *Natural Earth* data. 
* *Create hexagon tiles*: MMQGIS plugin offers the possibility to create hexagonal *grid layers*. 
* *Combine layers*: we obtain the desired hexagons by doing an intersection between the country layer and the grid layer.
* *Tile ID and coordinates*: we add to the table attributes of the grid layer and ID to each hexagonal tile and the coordinates of the center
* *Export hexagonal tiles*: we can export with MMQGIS a CSV file containing the table attributes

### Generate centers of hexagons

Centers are converted into coordinates suitable for **WebGL**. They are centered and divided by 1000.

In [452]:
centers = pd.read_csv("../qgis/hexagons_centers.csv", usecols = ['x','y','ID'])

In [453]:
means = centers.mean()

In [454]:
centers['x'] = (centers['x']-means['x'])/1000.0

In [455]:
centers['y'] = (centers['y']-means['y'])/1000.0

### Merge stations and hexagonal tiles

The stops with their coordinates (obtained from *https://transport.opendata.ch*)are imported in QGIS as as CSV file. We perform an intersection between the stops and the grid layer including attributes of both tables to get the tile ID for each stop. We can then export a file *stops_with_tile_ids.csv*.

## Create the JSON file for the stops

In [43]:
points = pd.read_csv("../qgis/stops_coordinates.csv", usecols = ['x','y','stop_id'])
stops_with_ids = pd.read_csv("../qgis/stops_with_tile_ids.csv")
points['x'] = (points['x'] - means['x'])/1000.0
points['y'] = (points['y'] - means['y'])/1000.0

In [44]:
points.head()

,x,y,stop_id
0,-40.742288,44.730223,132
1,30.257753,-70.469846,133
2,16.757734,-27.969834,134
3,-33.942253,-48.569779,135
4,-38.331283,-40.041801,136


In [45]:
temp = pd.merge(stops_with_ids, points, how='inner')

In [46]:
temp.head()

,stop_id,ID,x,y
0,132,553,-40.742288,44.730223
1,133,949,30.257753,-70.469846
2,134,892,16.757734,-27.969834
3,135,603,-33.942253,-48.569779
4,136,572,-38.331283,-40.041801


In [47]:
points = pd.merge(temp, centers[['ID', 'h']])

In [48]:
points.head()

,stop_id,ID,x,y,h
0,132,553,-40.742288,44.730223,7.6795
1,8500464,553,-41.928286,42.289236,7.6795
2,8500465,553,-40.906240,40.830184,7.6795
3,8500476,553,-42.618259,42.960168,7.6795
4,8500477,553,-42.679277,44.444202,7.6795


In [51]:
points.to_json("../res/stops.json", orient="records")

# Create JSON of Cities with geolocalisation and population

We need a JSON file containing the different cities of Switzerland with their respective population and geographical coordinates in order to display the most important ones on the map. 

The population file comes from the *Federal Statistical Office* and can be downloaded here: *https://www.bfs.admin.ch/bfs/en/home/statistics/population/surveys/statpop.html*. We need to get the coordinates of each city so we use the *GeoNames* server.

In [ ]:
cities = pd.read_csv("qgis/cities_population.csv")

In [28]:
cities.head()

,Commune,Total
0,0001 Aeugst am Albis,1970
1,0002 Affoltern am Albis,11494
2,0003 Bonstetten,5315
3,0004 Hausen am Albis,3450
4,0005 Hedingen,3642


In [36]:
from geopy.geocoders import GeoNames
geolocator = GeoNames(country_bias="Switzerland",username="ada_dream_team")

for i in cities.index[2001:]:
    newName = cities.ix[i,'Commune'][5:]
    location = geolocator.geocode(newName)
    if location:
        cities.ix[i,'x'] = location.longitude
        cities.ix[i,'y'] = location.latitude
        cities.ix[i,'Commune'] = newName
    else:
        cities.ix[i,'x'] = np.nan
        cities.ix[i,'y'] = np.nan

In [52]:
cities_complete = cities.dropna()

In [58]:
cities_complete.to_csv("map_js/cities_coordinates.csv", index=False)

### Inbetween we took the tile ID with QGIS, now we create the json 

In [28]:
c = pd.read_csv("map_js/cities_full.csv")

In [29]:
c.head()

,Commune,Total,x,y,ID
0,Aeugst am Albis,1970,8.48970,47.27541,906
1,Affoltern am Albis,11494,8.45128,47.27743,906
2,Bonstetten,5315,8.46625,47.31665,907
3,Hausen am Albis,3450,8.53299,47.24496,937
4,Hedingen,3642,8.44833,47.29794,906


In [30]:
import json
import unicodedata
cities_json = {}
for name, pop, tile_id, x , y in zip(c.Commune, c.Total, c.ID, c.x, c.y):
    if isinstance(name, str):
        n = str(unicodedata.normalize('NFD', name).encode('ascii', 'ignore'))[2:-1]
        cities_json[n] = {"population": str(pop), "ID": str(tile_id), "x":str(x),"y":str(y)}

In [31]:
cities_json

{'Rottenschwil': {'ID': '853',
  'population': '802',
  'x': '8.36137',
  'y': '47.31375'},
 'Bolken': {'ID': '553', 'population': '564', 'x': '7.66305', 'y': '47.19123'},
 'Hauptwil-Gottshaus': {'ID': '1310',
  'population': '1884',
  'x': '9.25066',
  'y': '47.48015'},
 'Calanca': {'ID': '1258',
  'population': '185',
  'x': '9.14886',
  'y': '46.26623'},
 'Bubikon': {'ID': '1071',
  'population': '6947',
  'x': '8.8179',
  'y': '47.26698'},
 'Bangerten': {'ID': '453',
  'population': '160',
  'x': '7.44914',
  'y': '47.05564'},
 'Wiggiswil': {'ID': '453',
  'population': '98',
  'x': '7.47086',
  'y': '47.03131'},
 'Hallwil': {'ID': '802',
  'population': '786',
  'x': '8.21577',
  'y': '47.28413'},
 'Autafond': {'ID': '264', 'population': '70', 'x': '7.07684', 'y': '46.8129'},
 'Corgemont': {'ID': '304',
  'population': '1651',
  'x': '7.14517',
  'y': '47.19457'},
 'Ringgenberg (BE)': {'ID': '643',
  'population': '2603',
  'x': '7.89445',
  'y': '46.70114'},
 'Schwerzenbach': {'I

In [32]:
json.dumps(cities_json)

'{"Rottenschwil": {"y": "47.31375", "population": "802", "ID": "853", "x": "8.36137"}, "Bolken": {"y": "47.19123", "population": "564", "ID": "553", "x": "7.66305"}, "Hauptwil-Gottshaus": {"y": "47.48015", "population": "1884", "ID": "1310", "x": "9.25066"}, "Calanca": {"y": "46.26623", "population": "185", "ID": "1258", "x": "9.14886"}, "Bubikon": {"y": "47.26698", "population": "6947", "ID": "1071", "x": "8.8179"}, "Bangerten": {"y": "47.05564", "population": "160", "ID": "453", "x": "7.44914"}, "Wiggiswil": {"y": "47.03131", "population": "98", "ID": "453", "x": "7.47086"}, "Hallwil": {"y": "47.28413", "population": "786", "ID": "802", "x": "8.21577"}, "Autafond": {"y": "46.8129", "population": "70", "ID": "264", "x": "7.07684"}, "Corgemont": {"y": "47.19457", "population": "1651", "ID": "304", "x": "7.14517"}, "Ringgenberg (BE)": {"y": "46.70114", "population": "2603", "ID": "643", "x": "7.89445"}, "Schwerzenbach": {"y": "47.38213", "population": "4947", "ID": "1007", "x": "8.65727

In [71]:
with open('map_js/cities.json', 'w') as outfile:
    json.dump(cities_json, outfile)

In [21]:
stops = pd.read_csv("../qgis/stops_with_tile_ids.csv")

In [22]:
stops.head()

,stop_id,ID
0,132,553
1,133,949
2,134,892
3,135,603
4,136,572


## Population tile-heights

In [456]:
from sklearn import preprocessing

In [457]:
stops_n_pop = pd.read_csv('../qgis/cities_full.csv')
grouped_pop = stops_n_pop.groupby('ID').agg({'Total': sum})

In [458]:
def counter(df):
    if df['ID'] in grouped_pop.index:
        return grouped_pop.loc[int(df['ID'])]['Total']
    else:
        return 0.0

In [459]:
centers['h'] = (centers.apply(counter, axis=1))

In [460]:
x = centers['h'].values #returns a numpy array 
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1,50))

# Root-scaling
root = lambda x: math.pow(x, 0.6)
x_rooted = centers.h.apply(root)
x_rooted = min_max_scaler.fit_transform(x_rooted)
centers['h'] = x_rooted

* x, y and ID are already in "qgis/hexagons_centers.csv". Need to add height h (affluence) and export JSON as "centers.json"
* need to do the same for the heights of stops: "qgis/stops_with_tile_ids.csv", add corresponding height of the tile

In [461]:
centers.to_csv("../res/centers.csv", index=False)

In [462]:
centers.to_json("../res/centers.json",orient="records")